In [1]:
import sys
sys.path.append("../")
print(sys.path)
from file_utils import load_jsonl, save_jsonl


['/home/afreens/ragged', '/home/afreens/KILT', '/home/afreens/miniconda3/envs/vlr/lib/python38.zip', '/home/afreens/miniconda3/envs/vlr/lib/python3.8', '/home/afreens/miniconda3/envs/vlr/lib/python3.8/lib-dynload', '', '/home/afreens/miniconda3/envs/vlr/lib/python3.8/site-packages', '../']


In [2]:
colbert_dataset_map = {
    "nq": "/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/predictions/colbert/nq-dev-kilt.jsonl",
    "bioasq": "/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/predictions/colbert/complete_bioasq.jsonl",
    "hotpotqa": "/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/predictions/colbert/hotpotqa-dev-kilt.jsonl"
}

colbert_dataset_map = {
    "nq": "nq-dev-kilt.jsonl",
    "bioasq": "complete_bioasq.jsonl",
    "hotpotqa": "hotpotqa-dev-kilt.jsonl"
}

In [3]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

INSTRUCTION_STR = "Give simple short one phrase answers for the questions based on the context"
NO_CONTEXT_INSTRUCTION_STR = "Give simple short one phrase answers for the question"
enc_len = len(enc.encode(INSTRUCTION_STR))
print(enc_len)
enc_len = len(enc.encode(NO_CONTEXT_INSTRUCTION_STR))
print(enc_len)


13
9


In [5]:
import os

from ragged.reader.utils import merge_retriever_data_and_eval_results
BASE_FOLDER = "/data/tir/projects/tir6/general/afreens/dbqa"
RETRIEVER_FOLDER = f"{BASE_FOLDER}/retriever_results"


write_folder = "/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count"
for dataset, path in bm25_dataset_map.items():

    retriever_data_path = path
    retriever_eval_path = os.path.join(RETRIEVER_FOLDER, "evaluations","bm25", dataset_map[dataset])
    print(retriever_data_path)
    print(retriever_eval_path)
    retriever_data = merge_retriever_data_and_eval_results(retriever_data_path, retriever_eval_path)
    # data = load_jsonl(path)
    token_data = []
    for d in retriever_data:
        instance_id = d["id"]
        question = d["input"]
        outputs = d["output"][0]["provenance"]
        output_tokens = [len(enc.encode(o["text"])) for o in outputs[:50]]
        par_id_match = [o["wiki_par_id_match"] for o in outputs[:50]]
        token_data.append({
            "instance_id": instance_id,
            "question_tokens" : len(enc.encode(question)),
            "output_tokens" : output_tokens,
            "par_id_match": par_id_match
        })
    save_jsonl(token_data, f"{write_folder}/{dataset}.jsonl")
            

NameError: name 'bm25_dataset_map' is not defined

In [12]:


for dataset in colbert_dataset_map:
    top_k = {}
    top_pos = {}
    no_context = 0
    path = f"{write_folder}/colbert/{dataset}.jsonl"
    print(path)
    data = load_jsonl(path, sort_by_id=False)
    for k in [0,1,2,3, 5, 10, 20, 30, 50]:
        # print(k)
        # print([x["output_tokens"][:k] for x in data])
        top_k[k] = sum([sum(x["output_tokens"][:k]) for x in data])
        top_k[k] += sum([x["question_tokens"] for x in data])
        top_k[k] += len(data) * 9 if k==0 else len(data) * 13
        if k>0:
            total = 0
            for x in data:
                positive_data = []
                for o, p in zip(x["output_tokens"], x["par_id_match"]):
                    if p:
                        positive_data.append(o)
                total+= (sum(positive_data[:k]) + x["question_tokens"])
            top_pos[k] = total + len(data)*13

    print(dataset)
    print("top_k", top_k)
    print("top_pos", top_pos)

    print("output_tokens", len(data)*10)
            
                


/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/nq.jsonl
loading from /data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/nq.jsonl
nq
output_tokens 28370
/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/bioasq.jsonl
loading from /data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/bioasq.jsonl
bioasq
output_tokens 38370
/data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/hotpotqa.jsonl
loading from /data/tir/projects/tir6/general/afreens/dbqa/retriever_results/token_count/colbert/hotpotqa.jsonl
hotpotqa
output_tokens 56000
